# Using Sentinel-2 Data to Calculate Burned Area

Welcome to the second project in the Earth Observation series. In this project, we will use the Sentinel-2 satellite data to calculate the burned area of Maui from the 2023 fires.

## Learning Objectives

In this project you will:

- Familiarize yourself with Sentinel-2 satellite data.
- Learn how to calculate the burned area using the Normalized Burn Ratio (NBR) index.
- Apply the NBR index to the Sentinel-2 data to calculate the burned area of Maui from the 2023 fires.

## Analysis Approach

Sentinel-2 is a European Space Agency (ESA) satellite that provides multispectral data at a high spatial resolution of 10 meters. Sentinel-2 revisits the same location every 5 days, which makes it a great tool for monitoring changes on the Earth's surface.

The Normalized Burn Ratio (NBR) is a widely used index to calculate the burned area from satellite data. The NBR index is calculated using the Near-Infrared (NIR) and Shortwave Infrared (SWIR2) bands of the satellite data. The formula to calculate the NBR index is:

NBR = (NIR - SWIR2) / (NIR + SWIR2)

The NBR index ranges from -1 to 1. To quantify burn severity, we will be using the delta NBR (dNBR) index, which is calculated as the difference between the pre-fire NBR and the post-fire NBR. The formula to calculate the dNBR index is:

dNBR = NBR_pre - NBR_post

## Let's Get Started!

The required libraries will be listed in the next cell for you to install if you don't have them already.

In [73]:
!pip install -q -U earthengine-api geemap

The following cell will import and authenticate the Google Earth Engine API.

In [74]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Retrieve and Process Pre- and Post-Fire Sentinel-2 Data

The first step in calculating the burned area is to retrieve the Sentinel-2 data for the pre- and post-fire periods. We will use the Sentinel-2 harmonized surface reflectance data for this analysis which contains additional bands with Scene Classification information.

### Bounding Box for Maui

We will use the TIGER/2018/Countries dataset to get the bounding box for Maui (county), Hawaii. The bounding box will be used to filter the Sentinel-2 data for Maui.

### High-Quality Sentinel-2 Data

To generate pre- and post-fire images for Maui, we will want to calculate NBR indices from images without cloud cover. As discussed in the previous project, one option is to use the very rudimentary filter of `CLOUDY_PIXEL_PERCENTAGE` when requesting images from Google Earth Engine. In this case, we will try to create a more sophisticated image from which we can calculate the NBR index by using the `qualityMosaic` function provided by Google Earth Engine.

#### Quality Mosaic

The `qualityMosaic` function in Google Earth Engine selects pixel values based on the highest quality score from a specified band, creating a composite image that prioritizes data quality.

Sentinel-2 data has a cloud score dataset called CLOUD_SCORE_PLUS. For Sentinel-2 harmonized surface reflectance data, the CLOUD_SCORE_PLUS dataset has two band, [which you can read about here](https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_CLOUD_SCORE_PLUS_V1_S2_HARMONIZED). The `cs` band in the CLOUD_SCORE_PLUS dataset will be used in the `qualityMosaic` function to prioritize cloud-free pixels.

#### Scene Classification

The Sentinel-2 harmonized surface reflectance data also contains a Scene Classification band that provides information about the land cover type of each pixel. We will use this band to filter out pixels that are not vegetation or bare soil when calculating the NBR index.

The goal is for the `qualityMosaic` function to return images that are cloud-free and contain only vegetation and bare soil pixels. However, the use of the Scene Classification Layer mask will end up filtering out cloudy pixels leading to holes in the image. We will use the `fill` function to fill in these holes with data from other images.

### Instructions

1. Retrieve the bounding box for Maui, Hawaii using the TIGER/2018/Countries dataset.
2. Implement the function to mask out non-vegetation and non-bare soil pixels using the Scene Classification band.
3. Implement the function to get the cloud score band from the CLOUD_SCORE_PLUS dataset.
4. Implement the function to filter out cloudy pixels using the `qualityMosaic` function.

In [75]:
# Use GEE to get the extend of Maui
roi = ()


def mask_clouds_sentinel2(image: ee.Image) -> ee.Image:
    """
    Function to mask clouds using the Scene Classification Layer (SCL) in Sentinel-2 SR data.

    See bands here: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#bands
    """
    # Use scene classification band
    scl = image.select("SCL")

    # Only keep vegetation (4) or bare soil (5)

    # Apply mask and normalize image
    return image.updateMask(scl_mask).divide(10000)


def get_cloud_score(roi: ee.Geometry, start_date: str, end_date: str) -> ee.Image:
    """
    Return the cloud score image collection for the given date range and region of interest.

    See: https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_CLOUD_SCORE_PLUS_V1_S2_HARMONIZED

    The cloud score is a value between 0 and 1, where 0 is not clear and 1 is clear.
    """
    pass


def get_quality_mosaic(roi: ee.Geometry, start_date: str, end_date: str) -> ee.Image:
    """
    With the given date range, this function returns a mosaic image that is mostly
    cloud-free and has the highest cloud score value.

    The images are also masked for anything that is not vegetation or bare soil.
    """
    cloud_score = get_cloud_score(roi, start_date, end_date)
    return mask_clouds_sentinel2(
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterBounds(roi)
        .filterDate(start_date, end_date)
        .linkCollection(cloud_score, "cs")  # Link cloud score to image collection
        .qualityMosaic("cs")  # Use cs to avoid clouds
        .clip(roi)
    )


# Pre-fire and post-fire image collections for cloud-free mosaic images
pre_fire = get_quality_mosaic(roi, "2023-07-01", "2023-08-07")
post_fire = get_quality_mosaic(roi, "2023-08-15", "2023-10-30")

The next cell will be used to show the resulting images for the pre- and post-fire periods.

In [76]:
import geemap

# Add layers to map for visualization
Map = geemap.Map()

# Center the map on the ROI
Map.centerObject(roi, 9)

# Store visualization parameters for RGB
rgb_vis = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 0.3}

# Add pre- and post-fire layers
Map.addLayer(pre_fire, rgb_vis, "Pre-Fire Image")
Map.addLayer(post_fire, rgb_vis, "Post-Fire Image")

# Display the map
Map

Map(center=[20.867841344178434, -156.61650489232312], controls=(WidgetControl(options=['position', 'transparen…

## Reflection

Take a look through the two layers we added to the map. Do you notice the holes left by the cloud masking? Even so, the image quality looks much better than the previous project. The next step is to calculate the NBR index for the pre- and post-fire images.

### Calculate the NBR Index

Using the pre- and post-fire images, we will calculate the NBR index for each image. The NBR index is calculated using the Near-Infrared (NIR) and Shortwave Infrared (SWIR2) bands of the Sentinel-2 data.

Using the NBR index, we can calculate the delta NBR (dNBR) index, which is the difference between the pre-fire NBR and the post-fire NBR. The dNBR index is used to quantify burn severity.

### Instructions

1. Implement the function to calculate the NBR index using the Near-Infrared (NIR) and Shortwave Infrared (SWIR2) bands.
2. Calculate the delta NBR (dNBR) index using the pre- and post-fire NBR indices.
3. Mask low-burn severity areas by excluding values that fall below a 0.1 threshold.

In [77]:
def calculate_nbr(image: ee.Image) -> ee.Image:
    """
    Function to calculate the Normalized Burn Ratio (NBR) using Sentinel-2 bands.
    NBR = (B8 - B12) / (B8 + B12)
    """

    return image


# Calculate NBR for pre- and post-fire images
pre_nbr = calculate_nbr(pre_fire)
post_nbr = calculate_nbr(post_fire)

# Calculate Delta NBR (dNBR)

# Mask low-burn severity areas using a threshold of 0.1

Use the next cell to visualize the dNBR reesults.

In [78]:
dnbr_vis = {
    "min": 0,
    "max": 1,
    "palette": ["white", "pink", "red", "darkred", "black"],
}  # Burn severity palette

# Add dNBR layer
Map.addLayer(dnbr, dnbr_vis, "Delta NBR (Burn Severity)")

# Display the map
Map

Map(center=[20.867841344178434, -156.61650489232312], controls=(WidgetControl(options=['position', 'transparen…

## Reflection

Take a look at the dNBR image. The dNBR index is used to quantify burn severity, with higher values indicating higher burn severity. However, dNBR values can also be influenced by factors such as natural vegetation variability. [Take a look at the real locations of the fires.](https://en.wikipedia.org/wiki/2023_Hawaii_wildfires#/map/1)

Fortunately, the dNBR index captured the burned areas. In a future project, we will continue to refine the analysis to better classify areas affected by the fire and exclude areas with natural vegetation variability.